# Change Detection (CD) with Random Forest (RF)

### Post-Classification CD using RF classifier
### Using Google Earth Engine Python API and NICFI Normalized Analytic Basemap from December 2015, December 2017, December 2019, and December 2021

Author: Finn Geiger\
Date: April 6th 2023\
Contact:
- https://github.com/finn-geiger
- https://www.linkedin.com/in/finn-geiger-b1329a20b/

### Steps:
1. Importing the datasets and classification result of 2015
2. OPIT classification for 2017, 2019, and 2021
3. CD for TOI 2015 to 2017
4. CD for TOI 2017 to 2019
5. CD for TOI 2019 to 2021
6. Final products

### 1 Import and setup
#### 1.1 Importing the required libraries and packages

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import geemap
import ee
import os
import time
import pandas as pd
from tabulate import tabulate
#%pip install tabulate


The following classes and landcover IDs will be used:

In [2]:
info = {'Class name': ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies'],
        'landcover ID': [1, 2, 3, 4, 5, 6, 7]}

print(tabulate(info, headers='keys', tablefmt='fancy_grid'))

╒══════════════╤════════════════╕
│ Class name   │   landcover ID │
╞══════════════╪════════════════╡
│ Informal     │              1 │
├──────────────┼────────────────┤
│ Formal       │              2 │
├──────────────┼────────────────┤
│ Industrial   │              3 │
├──────────────┼────────────────┤
│ Roads        │              4 │
├──────────────┼────────────────┤
│ Vacant land  │              5 │
├──────────────┼────────────────┤
│ Vegetation   │              6 │
├──────────────┼────────────────┤
│ Water-bodies │              7 │
╘══════════════╧════════════════╛


##### When first using the GEE Python API the user must authenticate and initialize the environment by using the following two lines of codes:

In [3]:
#ee.Authenticate() 
#ee.Initialize()

In [4]:
# creating the map
Map = geemap.Map()

# loading the interactive map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#### 1.2 Importing the datasets from GEE assets and data catalog and clipping the basemap to the AOI

In [5]:
# importing the classified scene from 2017, 2019, and 2021
classified_TOI1_2015 = ee.Image('users/s85315/masterthesis/change_detection_results/classified_TOI1_RF')
classified_TOI2_2017 = ee.Image('users/s85315/masterthesis/change_detection_results/classified_TOI2_RF')
classified_TOI3_2019 = ee.Image('users/s85315/masterthesis/change_detection_results/classified_TOI3_RF')
classified_TOI4_2021 = ee.Image('users/s85315/masterthesis/change_detection_results/classified_TOI4_RF')

# creating the visualization parameters
palette = ['c43c39', 'e5b636', '2f2f2f', 'aaaaaa', 'b08e7a', '85b66f', 'a5bfdd']
vis_params_classified = {'min': 1, 'max': 7, 'palette': palette}

# Adding the thematic maps to the map
Map.centerObject(classified_TOI2_2017, 4)
Map.addLayer(classified_TOI1_2015, vis_params_classified, 'thematic map - 2015')
Map.addLayer(classified_TOI2_2017, vis_params_classified, 'thematic map - 2017')
Map.addLayer(classified_TOI3_2019, vis_params_classified, 'thematic map - 2019')
Map.addLayer(classified_TOI4_2021, vis_params_classified, 'thematic map - 2021')

In [6]:
# Loading the AOI and Masking the base scene
vis_params_aoi = {'color': 'blue'}
aoi_windhoek = ee.FeatureCollection('users/s85315/masterthesis/Study_Area_Windhoek')

# Adding the AOI to the map
Map.addLayer(aoi_windhoek, vis_params_aoi, 'AOI', False)
Map.centerObject(aoi_windhoek, 12)

In [27]:
# importing the change masks
change_mask_2015_2017 = ee.Image('users/s85315/masterthesis/change_detection_results/change_mask_2015_2017')
change_mask_2017_2019 = ee.Image('users/s85315/masterthesis/change_detection_results/change_mask_2017_2019')
change_mask_2019_2021 = ee.Image('users/s85315/masterthesis/change_detection_results/change_mask_2019_2021')
change_mask_2015_2021 = ee.Image('users/s85315/masterthesis/change_detection_results/change_mask_2015_2021')

# creating the visualization parameters
vis_params_changed = {'min':0, 'max':1, 'palette': ['white', 'red']}

Map.addLayer(change_mask_2015_2017, vis_params_changed, 'Change Mask - 2015/2017')
Map.addLayer(change_mask_2017_2019, vis_params_changed, 'Change Mask - 2017/2019')
Map.addLayer(change_mask_2019_2021, vis_params_changed, 'Change Mask - 2019/2021')
Map.addLayer(change_mask_2015_2021, vis_params_changed, 'Change Mask - 2015/2021')

### 2 Accuracy assessment for changed areas

#### 2.1 Importing validation samples from GEE Assets

In [28]:
# Importing merged validation samples for 2017, 2019, and 2021
vs_change_areas_2017 = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/ChangeDetection/VS_change_areas_TOI2_2017_RPoints')
vs_change_areas_2019 = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/ChangeDetection/VS_change_areas_TOI3_2019_RPoints')
# vs_change_areas_2021 = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/ChangeDetection/VS_change_areas_TOI4_2021_RPoints')

# Adding them to the map
Map.addLayer(vs_change_areas_2017, vis_params_aoi, 'Validation samples for changed areas - 2017')
Map.addLayer(vs_change_areas_2019, vis_params_aoi, 'Validation samples for changed areas - 2019')
# Map.addLayer(vs_change_areas_2021, vis_params_aoi, 'Validation samples for changed areas - 2021')


#### 2.2 Applying the validation samples to the thematic map

In [18]:
# applying the validation samples to the classified map
validation_ca_2017 = classified_TOI2_2017.sampleRegions(**{
  'collection': vs_change_areas_2017,
  'properties': ['landcover'],
  'tileScale': 16,
  'scale': 4.77,
  'projection': 'EPSG:32733'
})

In [29]:
validation_ca_2019 = classified_TOI3_2019.sampleRegions(**{
  'collection': vs_change_areas_2019,
  'properties': ['landcover'],
  'tileScale': 16,
  'scale': 4.77,
  'projection': 'EPSG:32733'
})

In [11]:
# validation_ca_2021 = classified_TOI4_2021.sampleRegions(**{
#   'collection': vs_change_areas_2021,
#   'properties': ['landcover'],
#   'tileScale': 16,
#   'scale': 4.77,
# })

#### 2.3 Generating the error matrix and printing information

##### For TOI 2 - 2017

In [20]:
TOI2_2017_error_matrix = validation_ca_2017.errorMatrix('b1', 'landcover')

# printing statistics
print('Confusion Matrix', TOI2_2017_error_matrix.getInfo())
print('Overall Accuracy', TOI2_2017_error_matrix.accuracy().getInfo())
print('Producers Accuracy', TOI2_2017_error_matrix.producersAccuracy().getInfo())
print('Consumers Accuracy', TOI2_2017_error_matrix.consumersAccuracy().getInfo())
print('Kappa', TOI2_2017_error_matrix.kappa().getInfo())

Confusion Matrix [[0, 0, 0, 0, 0, 0, 0, 0], [0, 21, 2, 4, 3, 5, 0, 0], [0, 3, 20, 3, 9, 1, 0, 0], [0, 2, 2, 23, 2, 0, 0, 0], [0, 4, 6, 0, 15, 0, 0, 0], [0, 0, 0, 0, 1, 24, 4, 6], [0, 0, 0, 0, 0, 0, 26, 0], [0, 0, 0, 0, 0, 0, 0, 4]]
Overall Accuracy 0.7
Producers Accuracy [[0], [0.6], [0.5555555555555556], [0.7931034482758621], [0.6], [0.6857142857142857], [1], [1]]
Consumers Accuracy [[0, 0.7, 0.6666666666666666, 0.7666666666666667, 0.5, 0.8, 0.8666666666666667, 0.4]]
Kappa 0.6446850393700787


In [30]:
TOI3_2019_error_matrix = validation_ca_2019.errorMatrix('b1', 'landcover')

# printing statistics
print('Confusion Matrix', TOI3_2019_error_matrix.getInfo())
print('Overall Accuracy', TOI3_2019_error_matrix.accuracy().getInfo())
print('Producers Accuracy', TOI3_2019_error_matrix.producersAccuracy().getInfo())
print('Consumers Accuracy', TOI3_2019_error_matrix.consumersAccuracy().getInfo())
print('Kappa', TOI3_2019_error_matrix.kappa().getInfo())

Confusion Matrix [[0, 0, 0, 0, 0, 0, 0, 0], [0, 23, 0, 6, 2, 0, 0, 0], [0, 0, 23, 1, 1, 0, 0, 0], [0, 1, 3, 23, 4, 1, 0, 0], [0, 5, 4, 0, 22, 5, 0, 0], [0, 1, 0, 0, 1, 24, 1, 5], [0, 0, 0, 0, 0, 0, 28, 0], [0, 0, 0, 0, 0, 0, 1, 25]]
Overall Accuracy 0.8
Producers Accuracy [[0], [0.7419354838709677], [0.92], [0.71875], [0.6111111111111112], [0.75], [1], [0.9615384615384616]]
Consumers Accuracy [[0, 0.7666666666666667, 0.7666666666666667, 0.7666666666666667, 0.7333333333333333, 0.8, 0.9333333333333333, 0.8333333333333334]]
Kappa 0.7666666666666667


In [14]:
# TOI4_2021_error_matrix = validation_ca_2021.errorMatrix('classification', 'landcover')

# # printing statistics
# print('Confusion Matrix', TOI4_2021_error_matrix.getInfo())
# print('Overall Accuracy', TOI4_2021_error_matrix.accuracy().getInfo())
# print('Producers Accuracy', TOI4_2021_error_matrix.producersAccuracy().getInfo())
# print('Consumers Accuracy', TOI4_2021_error_matrix.consumersAccuracy().getInfo())
# print('Kappa', TOI4_2021_error_matrix.kappa().getInfo())

##### 4.3.1 Visualizing the error matrix and producer's and user's accuracy

##### 4.3.1.1 For 2015 to 2017

In [21]:
# creating a Pandas Dataframe for the error matrix
error_matrix = TOI2_2017_error_matrix.getInfo()
df_error_matrix = pd.DataFrame(error_matrix)

# deleting the first row and column since GEE add's a class with the landcover ID 0 by default.
df_error_matrix.columns = ['not used', 'Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
df_error_matrix = df_error_matrix.drop(df_error_matrix.columns[0],axis=1)
df_error_matrix.drop(index=df_error_matrix.index[0], axis=0, inplace=True)

# calculating row and column sum of points
column_total = df_error_matrix.sum()
column_total.name = 'Total'
df_error_matrix.loc[8] = column_total
df_error_matrix['Total'] = df_error_matrix.sum(axis=1)

header_error_matrix = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies', 'Total']
df_error_matrix['Names'] = header_error_matrix
df_error_matrix = df_error_matrix.set_index('Names')


print(tabulate(df_error_matrix, headers=header_error_matrix, tablefmt='fancy_grid', showindex=header_error_matrix))
df_error_matrix.to_csv("./data/accuracies/CD_Error_Matrix_TOI2.csv", sep=';', index=True)

# defining the variables
overall_accuracy = TOI2_2017_error_matrix.accuracy().getInfo()
overall_print = str(round(overall_accuracy * 100, 2))
kappa = TOI2_2017_error_matrix.kappa().getInfo()

df_overall_kappa = pd.DataFrame()

# printing out vaLues
print("\033[1m" + "Overall Accuracy " + overall_print + " %" + "\033[0m")
print("\033[1m" + "Kappa coefficent " + str(round(kappa, 2)) + "\033[0m")

╒══════════════╤════════════╤══════════╤══════════════╤═════════╤═══════════════╤══════════════╤════════════════╤═════════╕
│              │   Informal │   Formal │   Industrial │   Roads │   Vacant land │   Vegetation │   Water-bodies │   Total │
╞══════════════╪════════════╪══════════╪══════════════╪═════════╪═══════════════╪══════════════╪════════════════╪═════════╡
│ Informal     │         21 │        2 │            4 │       3 │             5 │            0 │              0 │      35 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Formal       │          3 │       20 │            3 │       9 │             1 │            0 │              0 │      36 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Industrial   │          2 │        2 │           23 │       2 │             0 │            0 │              0 │      29 │
├───────

In [22]:
# creating the lists 
producers = TOI2_2017_error_matrix.producersAccuracy().getInfo()
df_producers = pd.DataFrame(producers)
df_producers.drop(index=df_producers.index[0], axis=0, inplace=True)

class_names = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
df_producers['class names'] = class_names
df_producers.columns = ["Producer Accuracy", "Class name"]
df_producers['Producer Accuracy'] = df_producers['Producer Accuracy'].multiply(100).round(2)

# creating a dataframe from the list of consumer accuracies and remove landcover ID 0
consumers = TOI2_2017_error_matrix.consumersAccuracy().getInfo()
df_consumers = pd.DataFrame(consumers)
df_consumers = df_consumers.drop(df_consumers.columns[0],axis=1)
df_consumers.columns = class_names

# reshaping the dataframe from wide to long format:
df_consumers_long = pd.melt(df_consumers, var_name='Class name', value_name="Consumer Accuracy")
df_consumers_long = df_consumers_long[['Consumer Accuracy', 'Class name']]
df_consumers_long['Consumer Accuracy'] = df_consumers_long['Consumer Accuracy'].multiply(100).round(2)

# merging both dataframes
df_all = pd.merge(df_producers, df_consumers_long, on='Class name')
new_cols = ['Class name', 'Producer Accuracy', 'Consumer Accuracy']
df_all = df_all[new_cols]


print(tabulate(df_all, headers=["Class name", "Producer's Accuracy [%]", "Consumers's Accuracy [%]"], tablefmt='fancy_grid',  showindex=False))
df_all.to_csv("./data/accuracies/CD_Consumers_Producers_Accuracy_TOI2.csv", sep=";", index=False)

╒══════════════╤═══════════════════════════╤════════════════════════════╕
│ Class name   │   Producer's Accuracy [%] │   Consumers's Accuracy [%] │
╞══════════════╪═══════════════════════════╪════════════════════════════╡
│ Informal     │                     60    │                      70    │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Formal       │                     55.56 │                      66.67 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Industrial   │                     79.31 │                      76.67 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Roads        │                     60    │                      50    │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Vacant land  │                     68.57 │                      80    │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Vegetation   │                    10

##### 4.3.1.2 For 2017 to 2019

In [31]:
# creating a Pandas Dataframe for the error matrix
error_matrix = TOI3_2019_error_matrix.getInfo()
df_error_matrix = pd.DataFrame(error_matrix)

# deleting the first row and column since GEE add's a class with the landcover ID 0 by default.
df_error_matrix.columns = ['not used','Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
df_error_matrix = df_error_matrix.drop(df_error_matrix.columns[0],axis=1)
df_error_matrix.drop(index=df_error_matrix.index[0], axis=0, inplace=True)

# calculating row and column sum of points
column_total = df_error_matrix.sum()
column_total.name = 'Total'
df_error_matrix.loc[8] = column_total
df_error_matrix['Total'] = df_error_matrix.sum(axis=1)

header_error_matrix = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies', 'Total']
df_error_matrix['Names'] = header_error_matrix
df_error_matrix = df_error_matrix.set_index('Names')

# plotting the error matrix
print(tabulate(df_error_matrix, headers=header_error_matrix, tablefmt='fancy_grid', showindex=header_error_matrix))
df_error_matrix.to_csv("./data/accuracies/CD_Error_Matrix_TOI3.csv", sep=';', index=True)

# defining the variables
overall_accuracy = TOI3_2019_error_matrix.accuracy().getInfo()
overall_print = str(round(overall_accuracy * 100, 2))
kappa = TOI3_2019_error_matrix.kappa().getInfo()

df_overall_kappa = pd.DataFrame()

# printing out vaLues
print("\033[1m" + "Overall Accuracy " + overall_print + " %" + "\033[0m")
print("\033[1m" + "Kappa coefficent " + str(round(kappa, 2)) + "\033[0m")

╒══════════════╤════════════╤══════════╤══════════════╤═════════╤═══════════════╤══════════════╤════════════════╤═════════╕
│              │   Informal │   Formal │   Industrial │   Roads │   Vacant land │   Vegetation │   Water-bodies │   Total │
╞══════════════╪════════════╪══════════╪══════════════╪═════════╪═══════════════╪══════════════╪════════════════╪═════════╡
│ Informal     │         23 │        0 │            6 │       2 │             0 │            0 │              0 │      31 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Formal       │          0 │       23 │            1 │       1 │             0 │            0 │              0 │      25 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Industrial   │          1 │        3 │           23 │       4 │             1 │            0 │              0 │      32 │
├───────

In [32]:
# creating the lists 
producers = TOI3_2019_error_matrix.producersAccuracy().getInfo()
df_producers = pd.DataFrame(producers)
df_producers.drop(index=df_producers.index[0], axis=0, inplace=True)

class_names = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
df_producers['class names'] = class_names
df_producers.columns = ["Producer Accuracy", "Class name"]
df_producers['Producer Accuracy'] = df_producers['Producer Accuracy'].multiply(100).round(2)

# creating a dataframe from the list of consumer accuracies and remove landcover ID 0
consumers = TOI3_2019_error_matrix.consumersAccuracy().getInfo()
df_consumers = pd.DataFrame(consumers)
df_consumers = df_consumers.drop(df_consumers.columns[0],axis=1)
df_consumers.columns = class_names

# reshaping the dataframe from wide to long format:
df_consumers_long = pd.melt(df_consumers, var_name='Class name', value_name="Consumer Accuracy")
df_consumers_long = df_consumers_long[['Consumer Accuracy', 'Class name']]
df_consumers_long['Consumer Accuracy'] = df_consumers_long['Consumer Accuracy'].multiply(100).round(2)

# merging both dataframes
df_all = pd.merge(df_producers, df_consumers_long, on='Class name')
new_cols = ['Class name', 'Producer Accuracy', 'Consumer Accuracy']
df_all = df_all[new_cols]


print(tabulate(df_all, headers=["Class name", "Producer's Accuracy [%]", "Consumers's Accuracy [%]"], tablefmt='fancy_grid',  showindex=False))
df_all.to_csv("./data/accuracies/CD_Consumers_Producers_Accuracy_TOI3.csv", sep=";", index=False)

╒══════════════╤═══════════════════════════╤════════════════════════════╕
│ Class name   │   Producer's Accuracy [%] │   Consumers's Accuracy [%] │
╞══════════════╪═══════════════════════════╪════════════════════════════╡
│ Informal     │                     74.19 │                      76.67 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Formal       │                     92    │                      76.67 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Industrial   │                     71.88 │                      76.67 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Roads        │                     61.11 │                      73.33 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Vacant land  │                     75    │                      80    │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Vegetation   │                    10

##### 4.3.1.2 For 2019 to 2021

In [ ]:
# # creating a Pandas Dataframe for the error matrix
# error_matrix = TOI4_2021_error_matrix.getInfo()
# df_error_matrix = pd.DataFrame(error_matrix)

# # deleting the first row and column since GEE add's a class with the landcover ID 0 by default.
# df_error_matrix.columns = ['not used','Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
# df_error_matrix = df_error_matrix.drop(df_error_matrix.columns[0],axis=1)
# df_error_matrix.drop(index=df_error_matrix.index[0], axis=0, inplace=True)

# # calculating row and column sum of points
# column_total = df_error_matrix.sum()
# column_total.name = 'Total'
# df_error_matrix.loc[8] = column_total
# df_error_matrix['Total'] = df_error_matrix.sum(axis=1)

# header_error_matrix = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies', 'Total']
# df_error_matrix['Names'] = header_error_matrix
# df_error_matrix = df_error_matrix.set_index('Names')

# # plotting the results
# print(tabulate(df_error_matrix, headers=header_error_matrix, tablefmt='fancy_grid', showindex=header_error_matrix))
# df_error_matrix.to_csv("./data/accuracies/CD_Error_Matrix_TOI4.csv", sep=';', index=True)

# # defining the variables
# overall_accuracy = TOI4_2021_error_matrix.accuracy().getInfo()
# overall_print = str(round(overall_accuracy * 100, 2))
# kappa = TOI4_2021_error_matrix.kappa().getInfo()

# df_overall_kappa = pd.DataFrame()

# # printing out vaLues
# print("\033[1m" + "Overall Accuracy " + overall_print + " %" + "\033[0m")
# print("\033[1m" + "Kappa coefficent " + str(round(kappa, 2)) + "\033[0m")

╒══════════════╤════════════╤══════════╤══════════════╤═════════╤═══════════════╤══════════════╤════════════════╤═════════╕
│              │   Informal │   Formal │   Industrial │   Roads │   Vacant land │   Vegetation │   Water-bodies │   Total │
╞══════════════╪════════════╪══════════╪══════════════╪═════════╪═══════════════╪══════════════╪════════════════╪═════════╡
│ Informal     │         11 │        1 │            4 │       1 │             0 │            0 │              0 │      17 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Formal       │          3 │       19 │            1 │       3 │             0 │            0 │              0 │      26 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Industrial   │          6 │        6 │           23 │       1 │             0 │            0 │              0 │      36 │
├───────

In [ ]:
# # creating the lists 
# producers = TOI4_2021_error_matrix.producersAccuracy().getInfo()
# df_producers = pd.DataFrame(producers)
# df_producers.drop(index=df_producers.index[0], axis=0, inplace=True)

# class_names = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
# df_producers['class names'] = class_names
# df_producers.columns = ["Producer Accuracy", "Class name"]
# df_producers['Producer Accuracy'] = df_producers['Producer Accuracy'].multiply(100).round(2)

# # creating a dataframe from the list of consumer accuracies and remove landcover ID 0
# consumers = TOI4_2021_error_matrix.consumersAccuracy().getInfo()
# df_consumers = pd.DataFrame(consumers)
# df_consumers = df_consumers.drop(df_consumers.columns[0],axis=1)
# df_consumers.columns = class_names

# # reshaping the dataframe from wide to long format:
# df_consumers_long = pd.melt(df_consumers, var_name='Class name', value_name="Consumer Accuracy")
# df_consumers_long = df_consumers_long[['Consumer Accuracy', 'Class name']]
# df_consumers_long['Consumer Accuracy'] = df_consumers_long['Consumer Accuracy'].multiply(100).round(2)

# # merging both dataframes
# df_all = pd.merge(df_producers, df_consumers_long, on='Class name')
# new_cols = ['Class name', 'Producer Accuracy', 'Consumer Accuracy']
# df_all = df_all[new_cols]


# print(tabulate(df_all, headers=["Class name", "Producer's Accuracy [%]", "Consumers's Accuracy [%]"], tablefmt='fancy_grid',  showindex=False))
# df_all.to_csv("./data/accuracies/CD_Consumers_Producers_Accuracy_TOI4.csv", sep=";", index=False)

╒══════════════╤═══════════════════════════╤════════════════════════════╕
│ Class name   │   Producer's Accuracy [%] │   Consumers's Accuracy [%] │
╞══════════════╪═══════════════════════════╪════════════════════════════╡
│ Informal     │                     64.71 │                      36.67 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Formal       │                     73.08 │                      63.33 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Industrial   │                     63.89 │                      76.67 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Roads        │                     72.73 │                      53.33 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Vacant land  │                     65.91 │                      96.67 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Vegetation   │                     8

#### 2.4 Calulating the binary change / no change error matrix and printing information

In [23]:
# Importing binary change / no change validation samples for TOI1 to TOI2
validation_samples_TOI1_TOI2 = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/ChangeDetection/VS_TOI1_TOI2_RPoints')

validation_binary_TOI1_TOI2 = change_mask_2015_2017.sampleRegions(**{
  'collection': validation_samples_TOI1_TOI2,
  'properties': ['change'],
  'tileScale': 16,
  'scale': 4.77,
  'projection': 'EPSG:32733'
})

In [25]:
TOI1_TOI2_binary = validation_binary_TOI1_TOI2.errorMatrix('b1', 'change')

# printing statistics
print('Confusion Matrix', TOI1_TOI2_binary.getInfo())
print('Overall Accuracy', TOI1_TOI2_binary.accuracy().getInfo())
print('Producers Accuracy', TOI1_TOI2_binary.producersAccuracy().getInfo())
print('Consumers Accuracy', TOI1_TOI2_binary.consumersAccuracy().getInfo())
print('Kappa', TOI1_TOI2_binary.kappa().getInfo())

Confusion Matrix [[24, 1], [14, 54]]
Overall Accuracy 0.8387096774193549
Producers Accuracy [[0.96], [0.7941176470588235]]
Consumers Accuracy [[0.631578947368421, 0.9818181818181818]]
Kappa 0.6476382924981057


In [26]:
# creating a Pandas Dataframe for the error matrix
error_matrix = TOI1_TOI2_binary.getInfo()
df_error_matrix = pd.DataFrame(error_matrix)

# deleting the first row and column since GEE add's a class with the landcover ID 0 by default.
df_error_matrix.columns = ['Change', 'No Change']

# calculating row and column sum of points
column_total = df_error_matrix.sum()
column_total.name = 'Total'
df_error_matrix.loc[3] = column_total
df_error_matrix['Total'] = df_error_matrix.sum(axis=1)

header_error_matrix = ['Change', 'No Change', 'Total']
df_error_matrix['Names'] = header_error_matrix
df_error_matrix = df_error_matrix.set_index('Names')

# plotting the results
print(tabulate(df_error_matrix, headers=header_error_matrix, tablefmt='fancy_grid', showindex=header_error_matrix))
df_error_matrix.to_csv("./data/accuracies/CD_Change_NoChange_TOI1_TOI2.csv", sep=';', index=True)

# defining the variables
overall_accuracy = TOI1_TOI2_binary.accuracy().getInfo()
overall_print = str(round(overall_accuracy * 100, 2))
kappa = TOI1_TOI2_binary.kappa().getInfo()

df_overall_kappa = pd.DataFrame()

# printing out vaLues
print("\033[1m" + "Overall Accuracy " + overall_print + " %" + "\033[0m")
print("\033[1m" + "Kappa coefficent " + str(round(kappa, 2)) + "\033[0m")

╒═══════════╤══════════╤═════════════╤═════════╕
│           │   Change │   No Change │   Total │
╞═══════════╪══════════╪═════════════╪═════════╡
│ Change    │       24 │           1 │      25 │
├───────────┼──────────┼─────────────┼─────────┤
│ No Change │       14 │          54 │      68 │
├───────────┼──────────┼─────────────┼─────────┤
│ Total     │       38 │          55 │      93 │
╘═══════════╧══════════╧═════════════╧═════════╛
Overall Accuracy 83.87 %
Kappa coefficent 0.65


In [ ]:
# # Importing binary change / no change validation samples for TOI1 to TOI2
# validation_samples_TOI2_TOI3 = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/ChangeDetection/VS_TOI2_TOI3_RPoints')

# validation_binary_TOI2_TOI3 = change_mask_2017_2019.sampleRegions(**{
#   'collection': validation_samples_TOI2_TOI3,
#   'properties': ['change'],
#   'tileScale': 16,
#   'scale': 4.77,
# })

In [ ]:
# TOI2_TOI3_binary = validation_binary_TOI2_TOI3.errorMatrix('classification', 'change')

# # printing statistics
# print('Confusion Matrix', TOI2_TOI3_binary.getInfo())
# print('Overall Accuracy', TOI2_TOI3_binary.accuracy().getInfo())
# print('Producers Accuracy', TOI2_TOI3_binary.producersAccuracy().getInfo())
# print('Consumers Accuracy', TOI2_TOI3_binary.consumersAccuracy().getInfo())
# print('Kappa', TOI2_TOI3_binary.kappa().getInfo())

Confusion Matrix [[22, 5], [27, 39]]
Overall Accuracy 0.6559139784946236
Producers Accuracy [[0.8148148148148148], [0.5909090909090909]]
Consumers Accuracy [[0.4489795918367347, 0.8863636363636364]]
Kappa 0.3270013568521031


##### Resources for code snippets

https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/10_Export.ipynb \
https://worldbank.github.io/OpenNightLights/tutorials/mod6_6_RF_classifier.html \
https://towardsdatascience.com/how-to-easily-create-tables-in-python-2eaea447d8fd \
https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest